# Note: <br>The following file will fail if you will try to run it as it must contain private Spotify client settings.

First we had to install spotipy library by running `pip install spotipy`
<br>
According to https://anaconda.org/jkroes/spotipy

#### Using Spotify API request to approve few terms of use:<br>
* I understand that this app is not for commercial use
* I understand that I cannot migrate my app from non-commercial to commercial without permission
* I understand and agree with Spotify's Developer Terms of Service, Branding Guidelines, and Privacy Policy
    * https://developer.spotify.com/terms/
    * https://developer.spotify.com/branding-guidelines/
    * https://www.spotify.com/il-en/legal/privacy-policy/

Create access token and get a specific user liked songs (as a test).

In [438]:
from_year_d = 6
to_year_d = 9

import spotipy
import spotipy.util as util
import sys

import pandas as pd
import numpy as np
import re

username = ''
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost:8888/callback/'
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

NI BIEN NI MAL - Bad Bunny
One Level Down - Original mix - Sphera
Backseat Freestyle - Kendrick Lamar
Tusa - KAROL G
Ready To Let Go - Cage The Elephant
Diggin' a Hole - Downstairs Monsters
Starry Night - Original Mix - Peggy Gou
That's Life (feat. Mac Miller & Sia) - 88-Keys
Eleven - Khalid
Lalala - Y2K
מאושרים - Doli & Penn
ROXANNE - Arizona Zervas
Be Still - Liam Gallagher
לא חסר לי כלום - Avihu Pinchasov Rhythm Club
Sunday Best - Surfaces
Right Back (feat. A Boogie Wit Da Hoodie) - Khalid
I Got A Name - Stereo Version - Jim Croce
Lover Of The Light - Live From Red Rocks, Colorado - Mumford & Sons
What's the Use? - Mac Miller
Hard Sun - Eddie Vedder


**Next step:** Get audio features for songs who doesn't include in 'Billboard Year-End Hot 100 singles' (if a song that picked up will be part of the list, then he will be removed)<br>
Get 600 songs: 3 years, 200 each year.

* offset: The index of the first result to return. if you want to get the results with the start index 10 you will need to set the offset to 10.<br>
* limit: Maximum number of results to return.

In [439]:
songs_name_no={"2016": [],"2017": [], "2018":[]}
artists_name_no={"2016": [],"2017": [], "2018":[]}
track_id_no={"2016": [],"2017": [], "2018":[]}

for i in range(from_year_d, to_year_d): 
    for j in range(0,200,2):
        # spotify API for python doesn't work with multiple concatenation and NOT value
        # so had to take all the songs and eliminate the hebrew one later
        track_results = sp.search(q='year:201{}'.format(i), type='track', limit=2,offset=j)
        
        for j, k in enumerate(track_results['tracks']['items']):
            songs_name_no["201{}".format(i)].append(k['name'])
            artists_name_no["201{}".format(i)].append(k['artists'][0]['name'])
            track_id_no["201{}".format(i)].append(k['id'])

    print('Number of elements in 201{}_track_id list:'.format(i), len(track_id_no["201{}".format(i)]))

Number of elements in 2016_track_id list: 200
Number of elements in 2017_track_id list: 200
Number of elements in 2018_track_id list: 200


In [441]:
songs_df_no={"2016_df_no": pd.DataFrame(),"2017_df_no": pd.DataFrame(), "2018_df_no":pd.DataFrame()}

i = from_year_d
for key, value in songs_df_no.items():
    songs_df_no[key]['Title'] = songs_name_no["201{}".format(i)]
    songs_df_no[key]['Artist'] = artists_name_no["201{}".format(i)]
    songs_df_no[key]['id'] = track_id_no["201{}".format(i)]
    songs_df_no[key]['Year'] = "201{}".format(i)
    songs_df_no[key]['is_top100'] = 0
    i = i + 1
songs_df_no['2018_df_no'].head()

,Title,Artist,id,Year,is_top100
0,Falling,Trevor Daniel,4TnjEaWOeW0eKTKIEvJyCa,2018,0
1,לשוב הביתה,Ishay Ribo,52n4gF126eIllrGuc9Zus6,2018,0
2,Lucid Dreams,Juice WRLD,285pBltuF7vW8TeWk8hdRR,2018,0
3,לבחור נכון,Amir Dadon,7n6emXIcaECmkljP1rPlvQ,2018,0
4,אחת ולתמיד,Ishay Ribo,3bgNXXL7TjlBDOl36wLWHk,2018,0


<br>**Next step:**
* Eliminate songs in Hebrew in order to be more relevant to top100 list.<br>
Also our lyrics site doesn't include Hebrew songs.
* Note that there is a chance that Hebrew songs written in English letters will exist, we had to include because there is no way to determine their origin<br>
#### The reasons above forced us to take much bigger amount of songs in the first query as the elimination reduced it dramatically. 

In [442]:
i = from_year_d
for key, value in songs_df_no.items():
    print("Number of elements for 201{} with Hebrew songs: {}".format(i, len(songs_df_no[key]['Title'])))
    songs_df_no[key] = songs_df_no[key][~songs_df_no[key]['Title'].str.contains('[א-ת]', regex = True)]
    print("Number of elements for 201{} without Hebrew songs: {}\n".format(i, len(songs_df_no[key]['Title'])))
    i = i + 1

Number of elements for 2016 with Hebrew songs: 200
Number of elements for 2016 without Hebrew songs: 157

Number of elements for 2017 with Hebrew songs: 200
Number of elements for 2017 without Hebrew songs: 130

Number of elements for 2018 with Hebrew songs: 200
Number of elements for 2018 without Hebrew songs: 134



After we ran this cell we were able to determine the amount of Hebrew songs we had, and actually how many songs we have to 'work' with.<br><br>
Originally we query for 300 songs per year, after this step we reduced it to 200 per year.<br><br>
Assuming that some of them will also be remove once we will merge with the songs the part of top100, then amount of 130-160 is good.

<br><br>**Next step**: Concatenate between the 3 dataframes ignoring the indexes as all of them use indexes 0-199.

In [443]:
df_spotipy_no = pd.concat([songs_df_no['2016_df_no'], songs_df_no['2017_df_no'], songs_df_no['2018_df_no']],axis=0, sort=False, ignore_index=True)
mid = len(df_spotipy_no)/2
print("Shape of the dataset: {}".format(df_spotipy_no.shape))
df_spotipy_no.iloc[np.r_[0:2, mid:mid+2, -2:0]]

Shape of the dataset: (421, 5)


,Title,Artist,id,Year,is_top100
0,goosebumps,Travis Scott,6gBFPUFcJLzWGx4lenP6h2,2016,0
1,Say You Won't Let Go,James Arthur,0p6RzKrGeXzyYYd2RZPKd8,2016,0
210,hostage,Billie Eilish,1WsEgieHsWWndAzLkmV105,2017,0
211,Save That Shit,Lil Peep,0S0vWvyZ6Rc79TXkWxT9QA,2017,0
419,Faucet Failure,Ski Mask The Slump God,1ThmUihH9dF8EV08ku5AXN,2018,0
420,Thotiana,Blueface,3oh6SCCeLuXhFpEyepla6G,2018,0


<br>There are scenatious where same track get under multiple track IDs (single, as part of an album, etc).<br>
Therefore we are about to check it and correct if needed.

In [444]:
group = df_spotipy_no.groupby(['Artist','Title'], as_index=True).size()
print("The amount of duplicated songs: {}".format(group[group > 1].count()))

The amount of duplicated songs: 7


In [445]:
print("Songs count BEFORE drop duplicate: {}".format(len(df_spotipy_no)))
df_spotipy_no.drop_duplicates(subset=['Artist','Title'], inplace=True)
print("Songs count AFTER drop duplicate: {}".format(len(df_spotipy_no)))

Songs count BEFORE drop duplicate: 421
Songs count AFTER drop duplicate: 414


In [446]:
group = df_spotipy_no.groupby(['Artist','Title'], as_index=True).size()
print("The amount of duplicated songs: {}".format(group[group > 1].count()))

The amount of duplicated songs: 0


<br>**Next step:** Creat function go get Audio Features data so we can call it and use for each song.
#### The function must get dataframe thet conatain column with track id named 'id'.

In [447]:
# empty list, batchsize and the counter for None results
def getAudioFeatures(df):
    rows = []
    batchsize = 100
    None_counter = 0

    for i in range(0,len(df['id']),batchsize):
        batch = df['id'][i:i+batchsize]
        feature_results = sp.audio_features(batch)
        for i, t in enumerate(feature_results):
            if t == None:
                None_counter = None_counter + 1
            else:
                rows.append(t)
    print('Done,\nNumber of tracks where no audio features were available:',None_counter)
    return(rows)

In [448]:
rows = getAudioFeatures(df_spotipy_no)

Done,
Number of tracks where no audio features were available: 0


<br>**Next step:** Insert the audio features data collected into a NEW dataframe.

In [449]:
df_audio_features_no = pd.DataFrame.from_dict(rows, orient='columns')
print("Shape of the dataset: {}".format(df_audio_features_no.shape))
df_audio_features_no.head()

Shape of the dataset: (414, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0847,https://api.spotify.com/v1/audio-analysis/6gBF...,0.841,243837,0.728,6gBFPUFcJLzWGx4lenP6h2,0.000000,7,0.1490,-3.370,1,0.0484,130.049,4,https://api.spotify.com/v1/tracks/6gBFPUFcJLzW...,audio_features,spotify:track:6gBFPUFcJLzWGx4lenP6h2,0.430
1,0.6950,https://api.spotify.com/v1/audio-analysis/0p6R...,0.358,211467,0.557,0p6RzKrGeXzyYYd2RZPKd8,0.000000,10,0.0902,-7.398,1,0.0590,85.043,4,https://api.spotify.com/v1/tracks/0p6RzKrGeXzy...,audio_features,spotify:track:0p6RzKrGeXzyYYd2RZPKd8,0.494
2,0.1410,https://api.spotify.com/v1/audio-analysis/7MXV...,0.678,230453,0.588,7MXVkk9YMctZqd1Srtv4MB,0.000006,7,0.1370,-7.015,1,0.2760,186.005,4,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,audio_features,spotify:track:7MXVkk9YMctZqd1Srtv4MB,0.486
3,0.4140,https://api.spotify.com/v1/audio-analysis/7BKL...,0.748,244960,0.524,7BKLCZ1jbUBVqRi2FVlTVw,0.000000,8,0.1110,-5.599,1,0.0338,95.010,4,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,audio_features,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,0.661
4,0.7020,https://api.spotify.com/v1/audio-analysis/7MiZ...,0.391,131272,0.396,7MiZjKawmXTsTNePyTfPyL,0.405000,1,0.3150,-8.621,0,0.1890,99.112,5,https://api.spotify.com/v1/tracks/7MiZjKawmXTs...,audio_features,spotify:track:7MiZjKawmXTsTNePyTfPyL,0.199


In [450]:
df_audio_features_no = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset: {}".format(df_audio_features_no.shape))
df_audio_features_no.dtypes

Shape of the dataset: (414, 18)


acousticness        float64
analysis_url         object
danceability        float64
duration_ms           int64
energy              float64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
track_href           object
type                 object
uri                  object
valence             float64
dtype: object

<br>**Next step:** Processing the data - drop uneeded columns.

In [451]:
df_audio_features_no.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1,inplace=True)

print("Shape of the dataset: {}".format(df_audio_features_no.shape))
df_audio_features_no.dtypes

Shape of the dataset: (414, 14)


acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object

<br>**Next step:** Merge between audio features dataframe and our original dataframe.

In [452]:
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df_spotipy_final_no = pd.merge(df_spotipy_no, df_audio_features_no, on='id', how='inner')
print("Shape of the dataset: {}".format(df_spotipy_final_no.shape))
df_spotipy_final_no.head()

Shape of the dataset: (414, 18)


,Title,Artist,id,Year,is_top100,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,goosebumps,Travis Scott,6gBFPUFcJLzWGx4lenP6h2,2016,0,0.0847,0.841,243837,0.728,0.000000,7,0.1490,-3.370,1,0.0484,130.049,4,0.430
1,Say You Won't Let Go,James Arthur,0p6RzKrGeXzyYYd2RZPKd8,2016,0,0.6950,0.358,211467,0.557,0.000000,10,0.0902,-7.398,1,0.0590,85.043,4,0.494
2,Starboy,The Weeknd,7MXVkk9YMctZqd1Srtv4MB,2016,0,0.1410,0.678,230453,0.588,0.000006,7,0.1370,-7.015,1,0.2760,186.005,4,0.486
3,Closer,The Chainsmokers,7BKLCZ1jbUBVqRi2FVlTVw,2016,0,0.4140,0.748,244960,0.524,0.000000,8,0.1110,-5.599,1,0.0338,95.010,4,0.661
4,Devil Eyes,Hippie Sabotage,7MiZjKawmXTsTNePyTfPyL,2016,0,0.7020,0.391,131272,0.396,0.405000,1,0.3150,-8.621,0,0.1890,99.112,5,0.199


Note: No songs lost, we have the same number of enteries before & after the merge.

<br>Check if we have any duplication in track

In [453]:
df_spotipy_final_no[df_spotipy_final_no.duplicated(subset=['Artist','Title'],keep=False)]

,Title,Artist,id,Year,is_top100,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


<br><br>**Next step:** Get data for songs that are part of top100 Billboard playlists.<br><br>
**Basically the steps will be similar to how it been done on songs that are not part of top100 while the change is that we get our info from known playlists that already contain the top100 songs for each year.**

##### We had the option to include 'popularity' column for each song but as the popularity is updated ongoing data then an old song current popularity isn't relevant as we wishing to have his popularity according the same year he has been chosen - we marked those lines.

In [454]:
from pprint import pprint

pl_uris = ['spotify:playlist:2LWafCgWzsXGWv7wJeePjA', 
           'spotify:playlist:255aUSCuVTcdD5JTogG69d', 
           'spotify:playlist:37IRJrV9jd0LnsFTIY83ax'] # top 100 billboard singles playlists by order: 2016, 2017, 2018 

track_id_yes = {"2016": [],"2017": [], "2018":[]}
songs_name_yes = {"2016": [],"2017": [], "2018":[]}
artists_name_yes = {"2016": [],"2017": [], "2018":[]}


j = from_year_d
while True:
    for playlist in pl_uris:
        offset = 0
        tracks_id = sp.playlist_tracks(playlist, offset=offset,
                                      fields='items.track.id,total')
        songs_name = sp.playlist_tracks(playlist, offset=offset,
                                      fields='items.track.name.total')
        artist_name = sp.playlist_tracks(playlist, offset=offset,
                                      fields='items.track.artists.name.total')
    #     popularity = sp.playlist_tracks(playlist, offset=offset,
    #                                   fields='items.track.popularity.total')
        
        i = 0
        offset = offset + len(tracks_id['items'])
        
        for i in range(0, offset):
            if (tracks_id['items'][i].get('track').get('id') != None):
                track_id_yes['201{}'.format(j)].append(tracks_id['items'][i].get('track').get('id'))
                songs_name_yes['201{}'.format(j)].append(songs_name['items'][i].get('track').get('name'))
                artists_name_yes['201{}'.format(j)].append(artist_name['items'][i].get('track').get('artists')[0].get('name'))
            else:
                continue
        if (j < to_year_d-1):
            j = j + 1
        else:
            j = j + 1
            break
    if (j >= to_year_d):
        break

In [455]:
print("track_id_yes dictionery-dataframes size: {}, {}, {}".format(len(track_id_yes['2016']), len(track_id_yes['2017']), len(track_id_yes['2018'])))
print("songs_name_yes dictionery-dataframes size: {}, {}, {}".format(len(songs_name_yes['2016']), len(songs_name_yes['2017']), len(songs_name_yes['2018'])))
print("artists_name_yes dictionery-dataframes size: {}, {}, {}".format(len(artists_name_yes['2016']), len(artists_name_yes['2017']), len(artists_name_yes['2018'])))

track_id_yes dictionery-dataframes size: 98, 99, 100
songs_name_yes dictionery-dataframes size: 98, 99, 100
artists_name_yes dictionery-dataframes size: 98, 99, 100


We have missing data for 2 songs in top100 billboard 2016. It's data loss we have to absorb.

<br>**Next step:** Put the data collected from lists into data frames fer year and collect their audio features.<br>
Splitted into dataframe per year because getAudioFeatured function had to work with defined offset.

In [457]:
songs_df_yes = {"2016_df_yes": pd.DataFrame(),"2017_df_yes": pd.DataFrame(), "2018_df_yes":pd.DataFrame()}

i = from_year_d
for key, value in songs_df_yes.items():
    songs_df_yes[key]['Title'] = songs_name_yes["201{}".format(i)]
    songs_df_yes[key]['Artist'] = artists_name_yes["201{}".format(i)]
    songs_df_yes[key]['id'] = track_id_yes["201{}".format(i)]
    songs_df_yes[key]['Year'] = "201{}".format(i)
    songs_df_yes[key]['is_top100'] = 1
    i = i + 1
print("Shape of the dataset: {}".format(songs_df_yes['2016_df_yes'].shape))
songs_df_yes['2016_df_yes'].tail()

Shape of the dataset: (98, 5)


,Title,Artist,id,Year,is_top100
93,Humble And Kind,Tim McGraw,1qosWrKxri24ZIzH4ZDFcp,2016,1
94,Wicked,Future,6BbINUfGabVyiNFJpQXn3x,2016,1
95,Tiimmy Turner,Desiigner,0zMxL4BTjSqCsUtfdlcL8G,2016,1
96,See You Again (feat. Charlie Puth),Wiz Khalifa,7wqSzGeodspE3V6RBD5W8L,2016,1
97,Perfect,One Direction,3NLnwwAQbbFKcEcV8hDItk,2016,1


In [458]:
#Get the audio feature based on previous defined function.

audio_features_yes = {"2016": pd.DataFrame(),"2017": pd.DataFrame(), "2018":pd.DataFrame()}
#songs_df_yes['2016_df_yes']
for i in range(from_year_d, to_year_d):
    rows = getAudioFeatures(songs_df_yes['201{}_df_yes'.format(i)])
    audio_features_yes['201{}'.format(i)] = pd.DataFrame.from_dict(rows, orient='columns')
    print("Shape of audio features dataset 201{}: {}.".format(i, audio_features_yes['201{}'.format(i)].shape))

Done,
Number of tracks where no audio features were available: 0
Shape of audio features dataset 2016: (98, 18).
Done,
Number of tracks where no audio features were available: 0
Shape of audio features dataset 2017: (99, 18).
Done,
Number of tracks where no audio features were available: 0
Shape of audio features dataset 2018: (100, 18).


In [459]:
audio_features_yes['2017'].head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.581000,https://api.spotify.com/v1/audio-analysis/0FE9...,0.825,233713,0.652,0FE9t6xYkqWXU2ahLh6D8X,0.000000,1,0.0931,-3.183,0,0.0802,95.977,4,https://api.spotify.com/v1/tracks/0FE9t6xYkqWX...,audio_features,spotify:track:0FE9t6xYkqWXU2ahLh6D8X,0.931
1,0.229000,https://api.spotify.com/v1/audio-analysis/5CtI...,0.694,228827,0.815,5CtI0qwDJkDQGwXD1H1cLb,0.000000,2,0.0924,-4.328,1,0.1200,88.931,4,https://api.spotify.com/v1/tracks/5CtI0qwDJkDQ...,audio_features,spotify:track:5CtI0qwDJkDQGwXD1H1cLb,0.813
2,0.013000,https://api.spotify.com/v1/audio-analysis/0KKk...,0.853,206693,0.560,0KKkJNfGyhkQ5aFogxQAPU,0.000000,1,0.0944,-4.961,1,0.0406,134.066,4,https://api.spotify.com/v1/tracks/0KKkJNfGyhkQ...,audio_features,spotify:track:0KKkJNfGyhkQ5aFogxQAPU,0.860
3,0.000243,https://api.spotify.com/v1/audio-analysis/7ujx...,0.906,177000,0.625,7ujx3NYtwO2LkmKGz59mXp,0.000032,1,0.0975,-6.779,0,0.0903,150.018,4,https://api.spotify.com/v1/tracks/7ujx3NYtwO2L...,audio_features,spotify:track:7ujx3NYtwO2LkmKGz59mXp,0.423
4,0.030600,https://api.spotify.com/v1/audio-analysis/1dNI...,0.607,247627,0.649,1dNIEtp7AY3oDAKCGg2XkH,0.000025,11,0.1740,-6.695,0,0.0362,102.996,4,https://api.spotify.com/v1/tracks/1dNIEtp7AY3o...,audio_features,spotify:track:1dNIEtp7AY3oDAKCGg2XkH,0.505


<br>**Next step:** Concatenate between songs_df&audio_features each year.
* Because we might have duplicates (single, as part of an album, etc) and the merge must happen on the track id, decided to first merge between each year songs_df&audio_features dataframes and only then to concat the three of them.

In [460]:
# the 'inner' method will make sure that we only keep track IDs present in both datasets

dic_spotipy_final_yes = {"2016": pd.DataFrame(),"2017": pd.DataFrame(), "2018":pd.DataFrame()}

for i in range(from_year_d, to_year_d):
    dic_spotipy_final_yes['201{}'.format(i)] = pd.merge(songs_df_yes['201{}_df_yes'.format(i)],
                                                        audio_features_yes['201{}'.format(i)], 
                                                                           on='id', how='inner')
    print("Shape of the merged 201{} dataset: {}".format(i, dic_spotipy_final_yes['201{}'.format(i)].shape))

Shape of the merged 2016 dataset: (98, 22)
Shape of the merged 2017 dataset: (99, 22)
Shape of the merged 2018 dataset: (100, 22)


<br>**Next step:** Concatenate between the 3 dataframes ignoring the indexes.

In [461]:
df_spotipy_final_yes = pd.concat([dic_spotipy_final_yes['2016'], 
                                  dic_spotipy_final_yes['2017'], 
                                  dic_spotipy_final_yes['2018']],axis=0, sort=False, ignore_index=True)
mid = len(df_spotipy_final_yes)/2
print("Shape of the final dataset for songs in top100: {}".format(df_spotipy_final_yes.shape))
df_spotipy_final_yes.iloc[np.r_[0:2, mid:mid+2, -2:0]]

Shape of the final dataset for songs in top100: (297, 22)


,Title,Artist,id,Year,is_top100,acousticness,analysis_url,danceability,duration_ms,energy,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,Love Yourself,Justin Bieber,3hB5DgAiMAQ4DzYbsMq1IT,2016,1,0.83500,https://api.spotify.com/v1/audio-analysis/3hB5...,0.609,233720,0.378,...,0.280,-9.828,1,0.4380,100.418,4,https://api.spotify.com/v1/tracks/3hB5DgAiMAQ4...,audio_features,spotify:track:3hB5DgAiMAQ4DzYbsMq1IT,0.515
1,Sorry,Justin Bieber,69bp2EbF7Q2rqc5N3ylezZ,2016,1,0.07970,https://api.spotify.com/v1/audio-analysis/69bp...,0.654,200787,0.760,...,0.299,-3.669,0,0.0450,99.945,4,https://api.spotify.com/v1/tracks/69bp2EbF7Q2r...,audio_features,spotify:track:69bp2EbF7Q2rqc5N3ylezZ,0.410
148,Thunder,Imagine Dragons,0tKcYR2II1VCQWT79i5NrW,2017,1,0.00683,https://api.spotify.com/v1/audio-analysis/0tKc...,0.600,187147,0.810,...,0.155,-4.749,1,0.0479,167.880,4,https://api.spotify.com/v1/tracks/0tKcYR2II1VC...,audio_features,spotify:track:0tKcYR2II1VCQWT79i5NrW,0.298
149,T-Shirt,Migos,7KOlJ92bu51cltsD9KU5I7,2017,1,0.24200,https://api.spotify.com/v1/audio-analysis/7KOl...,0.865,242407,0.687,...,0.158,-3.744,0,0.2170,139.023,4,https://api.spotify.com/v1/tracks/7KOlJ92bu51c...,audio_features,spotify:track:7KOlJ92bu51cltsD9KU5I7,0.486
295,Mi Gente (feat. Beyoncé),J Balvin,0GzmMQizDeA2NVMUaZksv0,2018,1,0.02230,https://api.spotify.com/v1/audio-analysis/0Gzm...,0.759,209733,0.716,...,0.231,-6.360,0,0.0818,105.009,4,https://api.spotify.com/v1/tracks/0GzmMQizDeA2...,audio_features,spotify:track:0GzmMQizDeA2NVMUaZksv0,0.469
296,Believer,Imagine Dragons,0pqnGHJpmpxLKifKRmU6WP,2018,1,0.06220,https://api.spotify.com/v1/audio-analysis/0pqn...,0.776,204347,0.780,...,0.081,-4.374,0,0.1280,124.949,4,https://api.spotify.com/v1/tracks/0pqnGHJpmpxL...,audio_features,spotify:track:0pqnGHJpmpxLKifKRmU6WP,0.666


<br>**Next step:** Processing the data - drop uneeded columns.

**We check for duplicates only for our self known, as our data took from specific places and not by random search.<br>
When we have duplicated song, it's mean that this song won twice year and his duplicated data is important as his data have x2 value - his weight is doubled from regular song and it's right to keep it as is.**

In [462]:
group = df_spotipy_final_yes.groupby(['Artist','Title'], as_index=True).size()
print("The amount of duplicated songs: {}".format(group[group > 1].count()))

The amount of duplicated songs: 21


In [463]:
list(df_spotipy_final_yes.columns.values)

['Title',
 'Artist',
 'id',
 'Year',
 'is_top100',
 'acousticness',
 'analysis_url',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'time_signature',
 'track_href',
 'type',
 'uri',
 'valence']

In [464]:
df_spotipy_final_yes.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1,inplace=True)

print("Shape of the final dataset for songs in top100: {}".format(df_spotipy_final_yes.shape))
list(df_spotipy_final_yes.columns.values)

Shape of the final dataset for songs in top100: (297, 18)


['Title',
 'Artist',
 'id',
 'Year',
 'is_top100',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'time_signature',
 'valence']

In [465]:
df_spotipy_final_yes.iloc[np.r_[0:2, mid:mid+2, -2:0]]

,Title,Artist,id,Year,is_top100,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Love Yourself,Justin Bieber,3hB5DgAiMAQ4DzYbsMq1IT,2016,1,0.83500,0.609,233720,0.378,0.00,4,0.280,-9.828,1,0.4380,100.418,4,0.515
1,Sorry,Justin Bieber,69bp2EbF7Q2rqc5N3ylezZ,2016,1,0.07970,0.654,200787,0.760,0.00,0,0.299,-3.669,0,0.0450,99.945,4,0.410
148,Thunder,Imagine Dragons,0tKcYR2II1VCQWT79i5NrW,2017,1,0.00683,0.600,187147,0.810,0.21,0,0.155,-4.749,1,0.0479,167.880,4,0.298
149,T-Shirt,Migos,7KOlJ92bu51cltsD9KU5I7,2017,1,0.24200,0.865,242407,0.687,0.00,10,0.158,-3.744,0,0.2170,139.023,4,0.486
295,Mi Gente (feat. Beyoncé),J Balvin,0GzmMQizDeA2NVMUaZksv0,2018,1,0.02230,0.759,209733,0.716,0.00,11,0.231,-6.360,0,0.0818,105.009,4,0.469
296,Believer,Imagine Dragons,0pqnGHJpmpxLKifKRmU6WP,2018,1,0.06220,0.776,204347,0.780,0.00,10,0.081,-4.374,0,0.1280,124.949,4,0.666


<br>**Next step:** Merge between df_spotipy_final_yes&df_spotipy_final_no while making sure that if there is duplicates then to keep the entry that his top_100=0(true).

**Final step:** Save the new dataframe as csv for outer use.

In [ ]:
#df.to_csv('myname.csv')

In [466]:
df_spotipy_final_yes.to_csv('NOTtop100spotift.csv')
df_spotipy_final_no.to_csv('top100spotift.csv')